# Multi-Objective Capacitated VRP (MOVRP) Problem Using Multi-Objective Evolutionary Algorithms (MOEAs)

### Global Functions

In [1]:
import math
import numpy as np

def parse_vrp_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Strip whitespace and filter empty lines
    lines = [line.strip() for line in lines if line.strip()]

    # Parse metadata
    metadata = {}
    i = 0
    while not lines[i].startswith('NODE_COORD_SECTION'):
        if ':' in lines[i]:
            key, value = lines[i].split(':', 1)
            metadata[key.strip()] = value.strip().strip('"')
        i += 1

    # Parse node coordinates (1-based in file, but we'll make 0-based list)
    coords = []
    i += 1  # Skip section header
    while not lines[i].startswith('DEMAND_SECTION'):
        parts = lines[i].split()
        if len(parts) == 3:
            node_id = int(parts[0]) - 1  # Make 0-based
            x, y = int(parts[1]), int(parts[2])
            # Insert at correct index in case not sequential (though they usually are)
            while len(coords) <= node_id:
                coords.append(None)
            coords[node_id] = (x, y)
        i += 1

    # Parse demands (0-based list)
    demands = []
    i += 1  # Skip section header
    while not lines[i].startswith('DEPOT_SECTION'):
        parts = lines[i].split()
        if len(parts) == 2:
            node_id = int(parts[0]) - 1  # Make 0-based
            demand = int(parts[1])
            while len(demands) <= node_id:
                demands.append(None)
            demands[node_id] = demand
        i += 1

    # Parse depot(s) — usually just one, 1-based
    depots = []
    i += 1  # Skip section header
    while i < len(lines) and lines[i] != 'EOF':
        depot = int(lines[i])
        if depot == -1:
            break
        depots.append(depot - 1)  # Make 0-based
        i += 1

    # Compute distance matrix (Euclidean 2D)
    dimension = int(metadata.get('DIMENSION', len(coords)))
    dist_matrix = np.zeros((dimension, dimension))
    for a in range(dimension):
        for b in range(dimension):
            x1, y1 = coords[a]
            x2, y2 = coords[b]
            dist_matrix[a][b] = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

    return {
        'metadata': metadata,
        'coords': coords,  # List of (x, y) tuples, 0-based (index 0 = node 1)
        'demands': demands,  # List of integers, 0-based
        'depots': depots,  # List of 0-based indices (usually [0] for depot at node 1)
        'dist_matrix': dist_matrix  # Numpy array of floats
    }

# Example usage for your files
small_data = parse_vrp_file('data/small.vrp')
medium_data = parse_vrp_file('data/medium.vrp')
large_data = parse_vrp_file('data/large.vrp')

# Print some extracted info from large (based on the provided sample)
print(large_data['metadata'])
print("Depot index:", large_data['depots'][0])
print("Sample coords (first 5):", large_data['coords'][:5])
print("Sample demands (first 5):", large_data['demands'][:5])
print("Sample distance (node 0 to 1):", large_data['dist_matrix'][0][1])

{'NAME': 'X-n101-k25', 'COMMENT': 'Generated by Uchoa, Pecin, Pessoa, Poggi, Subramanian, and Vidal (2013)', 'TYPE': 'CVRP', 'DIMENSION': '101', 'EDGE_WEIGHT_TYPE': 'EUC_2D', 'CAPACITY': '206'}
Depot index: 0
Sample coords (first 5): [(365, 689), (146, 180), (792, 5), (658, 510), (461, 270)]
Sample demands (first 5): [0, 38, 51, 73, 70]
Sample distance (node 0 to 1): 554.1137067425782


## NSGA-II Implementation

In [ ]:
# TODO ADD  ALGO
import random
import copy


class NsgaII:
    """ 
    NSGA-II algorithm for Multi-Objective Capacitated VRP (MOVRP).
    Each individual is represented as a permutation of customers.
    """

    def __init__(self, data, pop_size=100, generations=100, crossover_rate=0.7, mutation_rate=0.2):
        self.data = data
        self.pop_size = pop_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.population = []

    def split_routes(self, customer_sequence):
        """
        Split the customer sequence into feasible routes based on vehicle capacity.
        :param customer_sequence: List of customer indices (excluding depot)
        :return: List of routes (each route is a list of node indices)
        """
        capacity = int(self.datea['metadata']['CAPACITY'])
        demands = self.data['demands']
        routes = []
        route = [0]
        load = 0
        for customer in customer_sequence:
            demand = demands[customer]
            if load + demand > capacity:
                route.append(0)
                routes.append(route)
                route = [0, customer]
                load = demand
            else:
                route.append(customer)
                load += demand
        route.append(0)
        routes.append(route)
        return routes
    
    def total_distance(self,routes):
        """
        Calculate the total distance of all routes.
        :param routes: List of routes 
        : return: Total distance(float)
        """
        dist_matrix = self.data['dist_matrix']
        return sum(
            sum(dist_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
            for route in routes
        )
    
    def max_route_length(self,routes):
        """ 
        Calculate the length of the longest route.
        :param routes: List of routes
        : return: Maximum route length(float)
        """
        dist_matrix = self.data[dist_matrix]
        return max(
            sum(dist_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
            for route in routes
        )
    
    def generate_initial_population(self):
        """ 
        Generate an initial population of random VRP individuals.
        :return: List of individuals (each is a list of customer indices)
        """
        n_customers = len(self.data['demands']) - 1
        population = []
        for _ in range(self.pop_size):
            perm = list(range(1, n_customers + 1))
            random.shuffle(perm)
            ind = {
                'customer_sequence': perm,
                'routes': self.split_routes(perm),
                'objectives': None,
                'rank': None,
                'crowding_distance': None
            }
            population.append(ind)
        return population



## SPEA2 Implementation

In [3]:
## TODO ADD ALGO